# Transfer Learning
We will try to implement face recognition using the pre-existing Mobile Net model.

In [93]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline


In [94]:
img_rows, img_cols = 224, 224

In [95]:
mobile=MobileNet(weights='imagenet',include_top=False,input_shape=(img_rows,img_cols,3))

In [96]:
# Here we freeze the last 4 layers
# Layers are set to trainable as True by default
for layer in mobile.layers:
    layer.trainable = False

In [97]:
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

In [98]:
# Configs
num_classes = 4
batch_size=5

## Modifying the MobileNet model

In [99]:
seqModel=Sequential()

for layer in mobile.layers:
    seqModel.add(layer)

# Adding custom Layers
seqModel.add(GlobalAveragePooling2D()) 
seqModel.add(Dense(1024,activation='relu')) #dense layer 1
seqModel.add(Dense(1024,activation='relu')) #dense layer 2
seqModel.add(Dense(512,activation='relu')) #dense layer 3
seqModel.add(Dense(num_classes ,activation='softmax')) #final layer with softmax activation

seqModel.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                          
                                                                 
 conv_dw_1_relu (ReLU)       (None, 112, 112, 32)     

In [100]:
train_path='/Coding/Learning/DL_Learning/Datasets/Facial_Recognition_TL/train'
valid_path='/Coding/Learning/DL_Learning/Datasets/Facial_Recognition_TL/valid'

In [101]:
train_batches=ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet.preprocess_input,
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
).flow_from_directory(
    train_path,
    target_size=(img_rows,img_cols),
    batch_size=batch_size,
    class_mode='categorical'
)

valid_batches=ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet.preprocess_input,
    rescale=1./255,
).flow_from_directory(
    valid_path,
    target_size=(img_rows,img_cols),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 237 images belonging to 4 classes.
Found 25 images belonging to 4 classes.


In [102]:
# # Just for Testing, Remove it.
# test_imgs, test_labels = next(train_batches)

# # Plotting the images
# def plotImages(images_arr):
#     fig, axes = plt.subplots(1, 10, figsize=(20,20))
#     axes = axes.flatten()
#     for img, ax in zip(images_arr, axes):
#         ax.imshow(img)
#         ax.axis('off')
#     plt.tight_layout()
#     plt.show()

# plotImages(test_imgs)

In [103]:
seqModel.compile(
    optimizer=RMSprop(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [104]:
curr_batch_size = 5
epochs=5

# Enter the number of training and validation samples here
nb_train_samples = 237
nb_validation_samples = 25

seqModel.fit(
    train_batches,
    steps_per_epoch=nb_train_samples//curr_batch_size,
    epochs=epochs,
    validation_data=valid_batches,
    validation_steps=nb_validation_samples//curr_batch_size,
    verbose=2
)

Epoch 1/5
47/47 - 25s - loss: 1.3532 - accuracy: 0.3879 - val_loss: 1.3413 - val_accuracy: 0.4000 - 25s/epoch - 530ms/step
Epoch 2/5
47/47 - 17s - loss: 1.3103 - accuracy: 0.4310 - val_loss: 1.3543 - val_accuracy: 0.4000 - 17s/epoch - 354ms/step
Epoch 3/5
47/47 - 17s - loss: 1.2280 - accuracy: 0.4914 - val_loss: 1.2631 - val_accuracy: 0.4000 - 17s/epoch - 360ms/step
Epoch 4/5
47/47 - 22s - loss: 1.2040 - accuracy: 0.4741 - val_loss: 1.2363 - val_accuracy: 0.4000 - 22s/epoch - 465ms/step
Epoch 5/5
47/47 - 17s - loss: 1.1443 - accuracy: 0.5172 - val_loss: 1.2213 - val_accuracy: 0.5200 - 17s/epoch - 352ms/step
